In [95]:
#from Adapt import AdaptVQE_Y

from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
import warnings
from qiskit_nature.circuit.library import HartreeFock,UCCSD
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator

from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE

from qiskit_nature.settings import QiskitNatureSettings
QiskitNatureSettings.dict_aux_operators = True

import numpy as np 
warnings.filterwarnings("ignore")

In [96]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

driver = PySCFDriver(atom="O 0 0 0;H 0.758602 0 0.504284;H 0.758602 0 -0.504284",basis="sto-3g")

problem = driver.run()
converter = QubitConverter(JordanWignerMapper())
#PN_property = properties.get_property("ParticleNumber")
#print(PN_property)

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)

#problem = ElectronicStructureProblem(driver, [transformer])
#second_q_ops = problem.second_q_ops()  # Get 2nd Quant OP
num_spin_orbitals = problem.num_spin_orbitals
num_particles = problem.num_particles
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print('轨道数目：{a}\n粒子数目{b}\n'.format(a=num_spin_orbitals,b=num_particles))

轨道数目：14
粒子数目(5, 5)



In [100]:
ansatz = UCCSD(num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,qubit_converter=converter,initial_state=init_state)
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = True # temporary fix

In [101]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(converter, adapt_vqe)

In [102]:
result = solver.solve(problem)

AttributeError: 'FermionicOp' object has no attribute 'to_list'

In [76]:

a = ansatz.to_instruction()
from qiskit.circuit import QuantumCircuit
qc= QuantumCircuit(4)
qc.append(a,[0,1,2,3])
qc.draw('mpl')


AttributeError: 'FermionicOp' object has no attribute 'to_list'

In [45]:
qc.decompose().decompose().draw()

┌───┐┌───────────────────────────────────┐»
q_0: ┤ X ├┤0                                  ├»
     └───┘│                                   │»
q_1: ─────┤1                                  ├»
     ┌───┐│  exp(-it (IIXY + IIYX))(1.0*t[0]) │»
q_2: ┤ X ├┤2                                  ├»
     └───┘│                                   │»
q_3: ─────┤3                                  ├»
          └───────────────────────────────────┘»
«     ┌───────────────────────────────────┐»
«q_0: ┤0                                  ├»
«     │                                   │»
«q_1: ┤1                                  ├»
«     │  exp(-it (XYII + YXII))(1.0*t[1]) │»
«q_2: ┤2                                  ├»
«     │                                   │»
«q_3: ┤3                                  ├»
«     └───────────────────────────────────┘»
«     ┌─────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                            ├
«     │                                                                             │
«q_1: ┤1                                                                            ├
«     │  exp(-it (YYXY + XXXY + XYYY + YXYY + XYXX + YXXX + YYYX + XXYX))(1.0*t[2]) │
«q_2: ┤2                                                                            ├
«     │                                                                             │
«q_3: ┤3                                                                            ├
«     └─────────────────────────────────────────────────────────────────────────────┘

In [46]:
vqe.initial_point

array([0., 0., 0.])

In [105]:
main_second_q_op, aux_second_q_ops = problem.second_q_ops()


In [62]:
problem.second_q_ops()

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j))], register_length=4, display_format='sparse'),
 'ElectronicEnergy': FermionicOp([('+_0 -_0', -1.2879605266732779), ('+_1 -_1', -0.5749964325186956), ('+_2 -_2', -1.2879605266732779), ('+_3 -_3', -0.5749964325186956), ('+_0 +_0 -_0 -_0', -0.4400795466875231), ('+_0 +_1 -_1 -_...)], register_length=4, display_format='sparse'),
 'DipoleMomentX': FermionicOp([('+_1 -_1', 1.0944875768738596), ('+_3 -_3', 1.0944875768738596)], register_length=4, display_format='sparse'),
 'DipoleMomentY': FermionicOp([('+_0 -_1', 0.04973035940289069), ('+_1 -_0', 0.04973035940289069), ('+_2 -_3', 0.04973035940289069), ('+_3 -_2', 0.04973035940289069)], register_length=4, display_format='sparse'),
 'DipoleMomentZ': FermionicOp([('', 0j)], register_length=4, display_format='sparse'),
 'AngularMomentum': FermionicOp([('+_0 -_0', (0.75+0j)), ('+_1 -_1', (0.75+0j)), ('+_2 -_2', (0.75+0j)), ('+_3 -_3'

In [68]:
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem2 = driver.run()
a =problem2.second_q_ops()